## This file is for evaluating the performance of the indexing module...

In [1]:
## Load the data

%load_ext autoreload
%autoreload 2

%pylab inline
import os
import cv2
import time
import numpy as np
import pandas as pd

import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image


print(os.getcwd())
home_dir = os.path.abspath('../../')
data_dir = os.path.join(home_dir, 'data', 'ua_detrac',  'DETRAC-Images')

filter_path = os.path.join(home_dir, 'filters')
loader_path = os.path.join(home_dir, 'loaders')

sys.argv=['']

sys.path.append(home_dir)
sys.path.append(loader_path)
sys.path.append(filter_path)

from loaders.uadetrac_loader import UADetracLoader
from eva_storage.preprocessingModule import PreprocessingModule
from eva_storage.UNet import UNet
from eva_storage.indexingModule import IndexingModule


Populating the interactive namespace from numpy and matplotlib
/home/jbang36/eva/eva_storage/evaluation


## Steps:
1. Load the dataset
2. Call the preprocessing module
3. Call the network
4. Call the indexing module
5. Do the evaluations done in file box_evaluation.ipynb


In [2]:
# 1. load the data  
loader = UADetracLoader()
images = loader.load_cached_images()
labels = loader.load_cached_labels()
boxes = loader.load_boxes()
video_start_indices = loader.get_video_start_indices()

['.ipynb_checkpoints', 'MVI_20011.xml', 'MVI_20012.xml', 'MVI_20032.xml', 'MVI_20033.xml', 'MVI_20034.xml', 'MVI_20035.xml', 'MVI_20051.xml', 'MVI_20052.xml', 'MVI_20061.xml', 'MVI_20062.xml', 'MVI_20063.xml', 'MVI_20064.xml', 'MVI_20065.xml']
skipping .ipynb_checkpoints
/home/jbang36/eva/data/ua_detrac/small-annotations/MVI_20011.xml
/home/jbang36/eva/data/ua_detrac/small-annotations/MVI_20012.xml
/home/jbang36/eva/data/ua_detrac/small-annotations/MVI_20032.xml
/home/jbang36/eva/data/ua_detrac/small-annotations/MVI_20033.xml
/home/jbang36/eva/data/ua_detrac/small-annotations/MVI_20034.xml
/home/jbang36/eva/data/ua_detrac/small-annotations/MVI_20035.xml
/home/jbang36/eva/data/ua_detrac/small-annotations/MVI_20051.xml
/home/jbang36/eva/data/ua_detrac/small-annotations/MVI_20052.xml
/home/jbang36/eva/data/ua_detrac/small-annotations/MVI_20061.xml
/home/jbang36/eva/data/ua_detrac/small-annotations/MVI_20062.xml
/home/jbang36/eva/data/ua_detrac/small-annotations/MVI_20063.xml
/home/jbang36

In [3]:
print(images.shape)

(10421, 300, 300, 3)


In [40]:
# 2. call the preprocessor
preprocessor = PreprocessingModule()
segmented_images = preprocessor.run(images, video_start_indices)

In [41]:
assert(images.shape[0] == segmented_images.shape[0]) ## number of datapoints are same
division_point = int(images.shape[0] * 0.8)
images_train = images[:division_point]
images_test = images[division_point:]
segmented_train = segmented_images[:division_point]
segmented_test = segmented_images[division_point:]


In [42]:
# 3. call the network
net = UNet()
# normally you would train, but I have checked that training works fine...so I will just load a frozen model
# you can load images by specifying a flag when you call train function and specify the epoch you want to load

net.train(images_train, segmented_train, load = False)
#net.train(images_train, segmented_train, load = True, epoch = 100)


New instance will be initialized
<class 'torch.device'>
Training the network....
epoch [1/100], loss:0.0989, time elapsed:129.4390 (sec)
Saving the trained model....
epoch [2/100], loss:0.0989, time elapsed:130.3971 (sec)
epoch [3/100], loss:0.0968, time elapsed:131.4944 (sec)
epoch [4/100], loss:0.0970, time elapsed:129.5675 (sec)
epoch [5/100], loss:0.0981, time elapsed:129.0419 (sec)
epoch [6/100], loss:0.0960, time elapsed:129.5399 (sec)
epoch [7/100], loss:0.0964, time elapsed:129.5178 (sec)
epoch [8/100], loss:0.0969, time elapsed:130.9213 (sec)
epoch [9/100], loss:0.0969, time elapsed:131.0785 (sec)
epoch [10/100], loss:0.0957, time elapsed:130.8821 (sec)
epoch [11/100], loss:0.0960, time elapsed:130.1001 (sec)
Saving the trained model....
epoch [12/100], loss:0.0958, time elapsed:130.4694 (sec)
epoch [13/100], loss:0.0958, time elapsed:128.8878 (sec)
epoch [14/100], loss:0.0959, time elapsed:133.5578 (sec)
epoch [15/100], loss:0.0955, time elapsed:138.7918 (sec)
epoch [16/100],

In [24]:
images_compressed, segmented_images = net.execute(images_test) 
## note I have overwritten variable segmented_images!!
## now segmented_images refer to output of the network


In [27]:
segmented_test = segmented_images
print(segmented_images.shape)
print(segmented_test.shape[0])
print(images_test.shape[0])
assert(segmented_test.shape[0] == images_test.shape[0])

(2085, 300, 300)
2085
2085


In [29]:
## fix the format of the image and segmented image array to uint8 because cv filters expect this format
images_test = images_test.astype(np.uint8)
segmented_test = segmented_test.astype(np.uint8)

In [30]:
# 4. call the indexing module - run post processsing methods
indexor = IndexingModule()
n_test_samples = images_test.shape[0]
proposed_boxes = []
for i in range(n_test_samples):
    tmp = images_test[i]
    tmp_seg = segmented_test[i]
    labels, results = indexor.post_individual(tmp_seg)
    assert(len(labels) == len(results))
    post = results[-1]
    patches = indexor.detect_patches(post)
    ## filter the boxes.. if they are too small or have bad aspect ratio take them out
    
    new_patches = indexor.filter_patches(patches)
    new_patches = indexor.reorder_patches(new_patches)
    
    proposed_boxes.append(new_patches)
    




In [34]:
## prepare the reference boxes
boxes_train = boxes[:division_point]
boxes_test = boxes[division_point:]
assert(len(boxes_test) == len(proposed_boxes))

In [32]:
import eva_storage.evaluation.utils as eval_utils

In [36]:
precision, recall = eval_utils.corloc(boxes_test, proposed_boxes, iou = 0.5)
print(precision, recall)

true positive 0
false positive 7865
false negative 32458
0.0 0.0


In [37]:
proposed_boxes

[[(49, 0, 64, 15), (16, 130, 32, 158), (2, 41, 29, 79), (0, 276, 22, 300)],
 [(49, 0, 64, 15), (16, 130, 32, 158), (2, 41, 29, 79), (0, 276, 22, 300)],
 [(49, 0, 64, 15), (16, 130, 32, 158), (2, 41, 29, 79), (0, 276, 22, 300)],
 [(49, 0, 64, 15), (16, 130, 32, 158), (2, 41, 29, 79), (0, 276, 22, 300)],
 [(49, 0, 64, 15), (16, 130, 32, 158), (2, 41, 29, 79), (0, 276, 22, 300)],
 [(49, 0, 64, 15), (16, 130, 32, 158), (2, 41, 29, 79), (0, 276, 22, 300)],
 [(49, 0, 64, 15), (16, 130, 32, 158), (2, 41, 29, 79), (0, 276, 22, 300)],
 [(49, 0, 64, 15), (16, 130, 32, 158), (2, 41, 29, 79), (0, 276, 22, 300)],
 [(49, 0, 64, 15), (16, 130, 32, 158), (2, 41, 29, 79), (0, 276, 22, 300)],
 [(49, 0, 64, 15), (16, 130, 32, 158), (2, 41, 29, 79), (0, 276, 22, 300)],
 [(49, 0, 64, 15), (16, 130, 32, 158), (2, 41, 29, 79), (0, 276, 22, 300)],
 [(49, 0, 64, 15), (16, 130, 32, 158), (2, 41, 29, 79), (0, 276, 22, 300)],
 [(49, 0, 64, 15), (16, 130, 32, 158), (2, 41, 29, 79), (0, 276, 22, 300)],
 [(49, 0, 64

In [38]:
boxes_test

array([list([(0, 212, 11, 220), (6, 199, 17, 205), (0, 203, 13, 208), (10, 209, 24, 217), (19, 222, 34, 232), (22, 218, 41, 230), (14, 166, 28, 174), (6, 179, 18, 185), (4, 167, 18, 176), (30, 263, 51, 291), (64, 264, 102, 288), (96, 44, 293, 155)]),
       list([(0, 213, 11, 220), (6, 199, 17, 205), (0, 203, 13, 208), (10, 209, 24, 217), (18, 222, 34, 232), (22, 218, 40, 230), (15, 166, 28, 174), (6, 179, 18, 185), (4, 167, 18, 176), (30, 263, 51, 290), (63, 263, 100, 287), (97, 42, 293, 156)]),
       list([(0, 213, 11, 220), (6, 199, 16, 205), (0, 203, 13, 208), (10, 209, 24, 217), (18, 222, 33, 232), (22, 218, 40, 229), (15, 167, 28, 174), (6, 179, 18, 185), (4, 168, 18, 176), (30, 262, 51, 290), (62, 262, 99, 286), (98, 40, 293, 156)]),
       ...,
       list([(42, 132, 68, 149), (53, 144, 79, 161), (6, 171, 20, 180), (99, 91, 295, 166), (4, 251, 46, 284)]),
       list([(43, 132, 68, 149), (52, 144, 79, 161), (6, 171, 20, 180), (100, 90, 295, 166), (4, 251, 46, 283)]),
       li

In [ ]:
## filter the recall results
filtered_test_boxes = filter_ground_truth(test_boxes)

precision, recall = corloc(filtered_test_boxes, proposed_boxes, iou = 0.5)